In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1615586826116_0007,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: org.apache.spark.SparkContext = org.apache.spark.SparkContext@5d382934


In [6]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.{col, element_at, split,monotonically_increasing_id,concat,lit}
import org.apache.spark.sql.types._

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.{col, element_at, split, monotonically_increasing_id, concat, lit}
import org.apache.spark.sql.types._


In [15]:
 def preProcessingDF(df: DataFrame):DataFrame = {
     val colsToDrop: List[String] = List("_c0","_c2","_c3","_c4","_c5","_c6","_c9","_c10","_c11","_c12",
                                         "_c13","_c14","_c15","_c16","_c17","_c18","_c19","_c20","_c21",
                                         "_c22","_c23","_c24","_c25","_c26","_c27","_c28","_c29","_c30","_c31")
     
        (df.drop(colsToDrop : _*)
         .withColumn("arrayHour", split(col("_c7")," "))
         .withColumn("Hour", element_at(col("arrayHour"),4))
         .drop("arrayHour","_c7")
         .withColumn("Date",concat(col("_c8"), lit(" "),col("Hour")))
         .withColumnRenamed("_c1","Tweets")
         .drop("Hour","_c8")
         .withColumn("ID", monotonically_increasing_id().cast(LongType))
         .withColumn("Date",col("Date").cast(TimestampType))
         .withColumn("Tweets",col("Tweets").cast(StringType))
         )
    }


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

preProcessingDF: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [3]:
val s3_path = "s3://trab4-bucket/datasets/debate-tweets.csv"

val df =   ( spark.read
  .option("inferSchema","true")
  .option("header","false")
  .option("sep","\t")
  .csv(s3_path) )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3_path: String = s3://trab4-bucket/datasets/debate-tweets.csv
df: org.apache.spark.sql.DataFrame = [_c0: bigint, _c1: string ... 30 more fields]


In [19]:
val df_processed = preProcessingDF(df)
df_processed.write.parquet("s3://trab4-bucket/datasets/debate-tweets.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_processed: org.apache.spark.sql.DataFrame = [Tweets: string, Date: timestamp ... 1 more field]
